# Gearshift test results

To facilitate the comparison of results between the MATLAB tool and the Python tool, we have created this notebook that reads the MATLAB and Python results and compares the output values.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Libraries that going to be used
import pandas as pd
from os.path import abspath, join, dirname
from pandas.util import hash_pandas_object

pd.set_option('display.max_rows', 125)

The downscaled factor (f_dsc) value can not be compared between Python and MATLAB because the value f_dsc in MATLAB is the calculated downscaling factor (this value is only used when, in the input file, the calc_dsc value is equal to 1). In Python, it represents the downscaling factor, therefore the two values are not the same.

In [3]:
column_rename = {# 'case': 'Case', 
' r_max': 'Required To Rated Power Ratio', 
#' f_dsc': 'Downscaling Factor', 
'   v_sum': 'Total Checksum', 
' v_max': 'Max Vehicle Speed', 
'  d_cycle': 'Total Distance',
'  g_avg': 'Average Gear Output', 
'  n_max1': 'Adjusted Max95 Engine Speed', 
'  n_max2': 'Max Engine Speed Cycle Output', 
'  n_max3': 'Max Engine Speed Reachable Output', 
'   n_max': 'Max Engine Speed Output',
' v_max_c': 'Max Vehicle Speed Cycle Output', 
' v_max_v': 'Max Vehicle Speed Reachable Output', 
' g_v_max': 'Gear Max Vehicle Speed Reachable Output', 
'  n_min1': 'Min Drive Engine Speed1st Output', 
' n_min12': 'Min Drive Engine Speed1st To2nd Output', 
' n_min2d': 'Min Drive Engine Speed2nd Decel Output',
'  n_min2': 'Min Drive Engine Speed2nd Output', 
'  n_min3': 'Min Drive Engine Speed Greater2nd Output',
' v_x_g_sum': 'Checksum Vx Gear Output'
} 

In [4]:
def highlight_false(val):
    color = 'red' if val == False else 'green'
    return 'background-color: %s' % color

def highlight_big_diff(val):
    color = 'red' if val > 0.2 else 'green'
    return 'background-color: %s' % color

def create_df_comparison_from_dict_results(results_dict, bool_comparison=True): 
    
    list_dfs = list()
    for key in results_dict.keys(): 
        _df = pd.DataFrame.from_dict(results_dict[key])
        
        if bool_comparison: 
            _df[key] = _df['python'] == _df['matlab']
        else:
            _df[key] = abs(_df['python'] - _df['matlab'])
        
        list_dfs.append(_df[[key]])
    
    if bool_comparison:
        return pd.concat(list_dfs, axis=1).T.style.applymap(highlight_false)
    else: 
        return pd.concat(list_dfs, axis=1).T.style.applymap(highlight_big_diff)

## Read Output Data

In [5]:
python_path = r'C:\Users\andre\Documents\European Comision\WLTP\_output\20210223_162455-'
matlab_path = r'C:\Users\andre\Documents\European Comision\WLTP\matlab_test\test_'

In [6]:
def df_round_summary(df_py):
    summary_py = pd.read_excel(df_py, 'Summary')
    summary_py['Required To Rated Power Ratio'] = round(summary_py['Required To Rated Power Ratio'], 3)
    summary_py['Adjusted Max95 Engine Speed'] = round(summary_py['Adjusted Max95 Engine Speed'], 2)
    summary_py['Max Engine Speed Cycle Output'] = round(summary_py['Max Engine Speed Cycle Output'], 2)
    summary_py['Max Engine Speed Reachable Output'] = round(summary_py['Max Engine Speed Reachable Output'], 2)
    

In [7]:
%%time

results = dict()

for case_no in range(1, 126):
    
    # python data
    df_py = pd.ExcelFile(python_path + str(case_no) + '.xlsx')
    summary_py = pd.read_excel(df_py, 'Summary')
    original_trace_py = pd.read_excel(df_py, 'Original Trace')
    applicable_trace_py = pd.read_excel(df_py, 'Applicable Trace')
    time_series_py = pd.read_excel(df_py, 'Time Series')
    available_powers_output_py = pd.read_excel(df_py, 'Available Powers Output')
    required_engine_speeds_output_py = pd.read_excel(df_py, 'Required Engine Speeds Output')
    
    # matlab data
    original_trace_ml = pd.read_csv(matlab_path + str(case_no) + '\OriginalTrace.txt', sep=",", header=None)
    original_trace_ml.rename(columns = {0: 'Trace Times', 1: 'Vehicle Speeds'}, inplace=True)
    applicable_trace_ml = pd.read_csv(matlab_path + str(case_no) + '\ApplicableTrace.txt', sep=",", header=None)
    applicable_trace_ml.rename(columns = {0: 'Trace Times', 1: 'Vehicle Speeds'}, inplace=True)
    available_powers_output_ml = pd.read_csv(matlab_path + str(case_no) + '\AvailablePowersOutput.txt', 
                                             sep=",", header=None)
    available_powers_output_ml = available_powers_output_ml.fillna(0)
    required_engine_speeds_output_ml = pd.read_csv(matlab_path + str(case_no) + '\RequiredEngineSpeedsOutput.txt', 
                                                   sep=",", header=None)
    required_engine_speeds_output_ml = required_engine_speeds_output_ml.fillna(0)
    gears_output_ml = pd.read_csv(matlab_path + str(case_no) + '\GearsOutput.txt', 
                                  sep=",", header=None)
    gears_corrections_output_ml = pd.read_csv(matlab_path + str(case_no) + '\GearCorrectionsOutput.txt', 
                                              sep=",", header=None)
    required_vehicles_speed_output_ml = pd.read_csv(matlab_path + str(case_no) + '\RequiredVehicleSpeedsOutput.txt',
                                                    sep=",", header=None)
    required_powers_output_ml = pd.read_csv(matlab_path + str(case_no) + '\RequiredPowersOutput.txt', 
                                            sep=",", header=None)
    clutch_disengaged_output_ml = pd.read_csv(matlab_path + str(case_no) + '\ClutchDisengagedOutput.txt', 
                                              sep=",", header=None)
    clutch_undefined_output_ml = pd.read_csv(matlab_path + str(case_no) + '\ClutchUndefinedOutput.txt', 
                                             sep=",", header=None)
    clutch_HST_output_ml = pd.read_csv(matlab_path + str(case_no) + '\ClutchHSTOutput.txt', 
                                       sep=",", header=None)
    clutch_HST_output_ml = clutch_HST_output_ml.T
    summary_ml = pd.read_csv(matlab_path + str(case_no) + '\case_result.txt', sep=",",)
    
    # append results
    results[case_no] = dict()
    
    results[case_no]['python'] = dict()
    results[case_no]['python']['summary'] = summary_py[list(column_rename.values())]
    results[case_no]['python']['original_trace_times'] = original_trace_py['Trace Times']
    results[case_no]['python']['original_trace_vehicle_speeds'] = original_trace_py['Vehicle Speeds']
    results[case_no]['python']['applicable_trace_times'] = applicable_trace_py['Trace Times']
    results[case_no]['python']['applicable_trace_vehicle_speeds'] = applicable_trace_py['Vehicle Speeds']
    results[case_no]['python']['ts_trace_times'] = time_series_py['Trace Times Output']
    results[case_no]['python']['ts_gears_out'] = time_series_py['Gears Output']
    time_series_py["Gear Corrections Output"] = \
                time_series_py["Gear Corrections Output"].apply(lambda x: x.replace('  ', ' '))
    results[case_no]['python']['ts_gears_correction_out'] = time_series_py["Gear Corrections Output"]
    results[case_no]['python']['ts_required_vehicle_speeds_out'] = time_series_py['Required Vehicle Speeds Output']
    results[case_no]['python']['ts_required_powers_out'] = time_series_py['Required Powers Output']
    results[case_no]['python']['ts_clutch_disengaged_out'] = time_series_py['Clutch Disengaged Output']
    results[case_no]['python']['ts_clutch_undefined_out'] = time_series_py['Clutch Undefined Output']
    results[case_no]['python']['ts_clutch_hst_out'] = time_series_py["Clutch H S T Output"]
    
    results[case_no]['matlab'] = dict()
    results[case_no]['matlab']['summary'] = summary_ml
    results[case_no]['matlab']['summary'].rename(columns=column_rename, inplace=True)
    results[case_no]['matlab']['summary'] = results[case_no]['matlab']['summary'][list(column_rename.values())]
    results[case_no]['matlab']['original_trace_times'] = original_trace_ml['Trace Times']
    results[case_no]['matlab']['original_trace_vehicle_speeds'] = original_trace_ml['Vehicle Speeds']
    results[case_no]['matlab']['applicable_trace_times'] = applicable_trace_ml['Trace Times']
    results[case_no]['matlab']['applicable_trace_vehicle_speeds'] = applicable_trace_ml['Vehicle Speeds']
    # results[case_no]['matlab']['ts_trace_times'] = 
    results[case_no]['matlab']['ts_gears_out'] = gears_output_ml[0]
    results[case_no]['matlab']['ts_gears_correction_out'] = gears_corrections_output_ml[0]
    results[case_no]['matlab']['ts_required_vehicle_speeds_out'] = required_vehicles_speed_output_ml[0]
    results[case_no]['matlab']['ts_required_powers_out'] = required_powers_output_ml[0]
    results[case_no]['matlab']['ts_clutch_disengaged_out'] = clutch_disengaged_output_ml[0]
    results[case_no]['matlab']['ts_clutch_undefined_out'] = clutch_undefined_output_ml[0]
    results[case_no]['matlab']['ts_clutch_hst_out'] = clutch_HST_output_ml[0]

Wall time: 1min 7s


## Create hashes and sums from Matlab and Python results

In [8]:
results_sums = dict()
results_hashes = dict()

variables = list(results[1]['python'].keys())

for case_no in range(1, 126): 
    results_sums[case_no] = dict(); results_hashes[case_no] = dict()
    results_sums[case_no]['python'] = dict(); results_hashes[case_no]['python'] = dict()
    results_sums[case_no]['matlab'] = dict(); results_hashes[case_no]['matlab'] = dict()
    for var in variables: 
        if var in list(results[case_no]['python'].keys()) and var in list(results[case_no]['matlab'].keys()):
            results_hashes[case_no]['matlab'][var] = results[case_no]['matlab'][var]
            results_hashes[case_no]['python'][var] = results[case_no]['python'][var]
                
            results_hashes[case_no]['matlab'][var] = \
                            hash_pandas_object(results_hashes[case_no]['matlab'][var]).sum()
            results_hashes[case_no]['python'][var] = \
                            hash_pandas_object(results_hashes[case_no]['python'][var]).sum()
            
            if var == 'summary': 
                for col in list(column_rename.values()): 
                    results_hashes[case_no]['matlab'][var + '-' + col] = \
                                    hash_pandas_object(results[case_no]['matlab'][var][col]).sum()
                    results_hashes[case_no]['python'][var + '-' + col] = \
                                    hash_pandas_object(results[case_no]['python'][var][col]).sum()
                
        
            if var not in ['summary']:
                if (results[case_no]['python'][var].dtypes == 'float64') or \
                                            (results[case_no]['python'][var].dtypes == 'int64'):

                    results_sums[case_no]['matlab'][var] = results[case_no]['matlab'][var].sum()
                    results_sums[case_no]['python'][var] = results[case_no]['python'][var].sum()
            
            if var in ['summary']:
                for col in list(column_rename.values()): 
                    results_sums[case_no]['matlab'][var + '-' + col] = \
                                    results[case_no]['matlab'][var][col].sum()
                    results_sums[case_no]['python'][var + '-' + col] = \
                                    results[case_no]['python'][var][col].sum()

### Hashes

In [9]:
create_df_comparison_from_dict_results(results_hashes, bool_comparison=True)

## Differences found in the columns:

* summary-Required To Rated Power Ratio: The maximum difference found is $5.551 * 10^{-17}$
* summary-Total Distance: The maximum difference found is $0$
* summary-Adjusted Max95 Engine Speed: The maximum difference found is $0$
* summary-Max Engine Speed Reachable Output: The maximum difference found is $0.01$, the difference is for the rounding done by Python and Matlab.
* summary-Max Engine Speed Output: The maximum difference found is $0$
* summary-Max Vehicle Speed Cycle Output: The maximum difference found is $0$
* summary-Max Vehicle Speed Cycle Output: The maximum difference found is $0$ 	
* summary-Max Vehicle Speed Reachable Output: The maximum difference found is $0$
* ts_required_vehicle_speeds_out: The maximum difference found is $2.84 * 10^{-14}$
* ts_required_powers_out: The maximum difference found is $2.22 * 10^{-16}$

In [10]:
create_df_comparison_from_dict_results(results_sums, bool_comparison=False)